In [2]:
import json
import openai
from dotenv import load_dotenv
import os
load_dotenv()



from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json
from utils.db import get_table_columns  # <-- make sure this exists in your utils.db



In [11]:
src_schema = 'int'
src_table = 'brand_master'
tgt_schema = 'dwh'
tgt_table = 'brand_master'

# Fetch schema metadata from DB
source_columns = get_table_columns(src_schema, src_table)
target_columns = get_table_columns(tgt_schema, tgt_table)

# Extract just the column names for prompt clarity
src_cols = [c["column_name"] for c in source_columns]
tgt_cols = [c["column_name"] for c in target_columns]

# Build LLM model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2,api_key=os.getenv('OPENAI_API_KEY'))

# Strict instruction to use only target columns
messages = [
    SystemMessage(content=(
        "You are an expert ETL engineer. "
        "Your task is to build a JSON Source-To-Target Mapping (STTM). "
        "You must only use the target columns that exist in the provided schema metadata. "
        "Do not invent or hallucinate new target columns. "
        "You may, however, infer transformations or use constants for audit fields like timestamps."

    )),
    HumanMessage(content=f"""
    Given:
    - Source table: `{src_schema}.{src_table}`
    - Target table: `{tgt_schema}.{tgt_table}`

    Source columns:
    {json.dumps(src_cols, indent=2)}

    Target columns:
    {json.dumps(tgt_cols, indent=2)}

    Generate an STTM JSON with entries like this:
    {{
        "target_column": "<column_name_in_target>",
        "source_columns": ["col1", "col2", ...],
        "transformation": "<expression or null>"
    }}

    Rules:
    - Map only columns present in the target list.
    - Use similar names for direct mappings (case-insensitive) except fou audit columns.
    - For audit fields like 'audit_insrt_dt', 'audit_updt_dt', 'load_ts', use CURRENT_TIMESTAMP.
    - Output must be valid JSON (list of mappings).
    -INT schema tables are always SCD1 for dimensions
    -DWH Schema tables are always SCD2 for dimensions.
    -Process_ind in the INT Schema will be I for new records & U for updated records & D for deleted records.
    -When the source schema is INT and target schema is DWH pick up records which have processind in I and U.
    -DWH tables will have is_active column which is a flag .Its set to Y for currently active records and N for inactive records.
    -EFF_START_DT & EFF_END_DT are the dates in between which the records were active/currently active.

    """)
]

response = model.invoke(messages)
reply = response.content.strip()

# Safe JSON extraction
try:
    sttm = json.loads(reply)
except Exception:
    import re
    match = re.search(r"\[[\s\S]*\]", reply)
    sttm = json.loads(match.group(0)) if match else []

# Double-check target column validity (safety filter)
valid_tgt_cols = set(tgt_cols)
filtered_sttm = [m for m in sttm if m.get("target_column") in valid_tgt_cols]
print(filtered_sttm)

c:\Users\A4627\OneDrive - Axtria\DSA_Python\Agentic_DataIngestion\utils\db.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
c:\Users\A4627\OneDrive - Axtria\DSA_Python\Agentic_DataIngestion\utils\db.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


[{'target_column': 'brand_id', 'source_columns': ['brand_id'], 'transformation': None}, {'target_column': 'brand_name', 'source_columns': ['brand_name'], 'transformation': None}, {'target_column': 'is_active', 'source_columns': ['process_ind'], 'transformation': "CASE WHEN process_ind = 'I' THEN 'Y' WHEN process_ind = 'U' THEN 'Y' ELSE 'N' END"}, {'target_column': 'eff_start_dt', 'source_columns': ['audit_insrt_dt'], 'transformation': None}, {'target_column': 'eff_end_date', 'source_columns': [], 'transformation': 'CURRENT_TIMESTAMP'}, {'target_column': 'audit_insrt_dt', 'source_columns': ['audit_insrt_dt'], 'transformation': None}, {'target_column': 'audit_insrt_nm', 'source_columns': ['audit_insrt_nm'], 'transformation': None}, {'target_column': 'audit_updt_dt', 'source_columns': ['audit_updt_dt'], 'transformation': 'CURRENT_TIMESTAMP'}, {'target_column': 'audit_updt_nm', 'source_columns': ['audit_updt_nm'], 'transformation': None}]


In [6]:
reply 


'Given that both the source and target tables have no specified columns, the JSON Source-To-Target Mapping (STTM) will also be empty. However, I will provide a template for the STTM based on the rules and structure you provided, assuming that there would be columns in a typical scenario.\n\nHere’s how the STTM JSON would look like if there were columns to map:\n\n```json\n[]\n```\n\nIf you had specific columns in mind for the source and target tables, please provide them, and I can create a more detailed mapping based on that information.'

In [7]:
source_columns

[]

In [ ]:
df=get_table_columns('int', '')
df

c:\Users\A4627\OneDrive - Axtria\DSA_Python\Agentic_DataIngestion\utils\db.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


[{'column_name': 'brand_id', 'data_type': 'text'},
 {'column_name': 'brand_name', 'data_type': 'text'},
 {'column_name': 'process_ind', 'data_type': 'text'},
 {'column_name': 'audit_insrt_dt', 'data_type': 'timestamp without time zone'},
 {'column_name': 'audit_insrt_nm', 'data_type': 'text'},
 {'column_name': 'audit_updt_dt', 'data_type': 'timestamp without time zone'},
 {'column_name': 'audit_updt_nm', 'data_type': 'text'},
 {'column_name': 'audit_batch_id', 'data_type': 'text'}]